In [116]:
import re

def clean_text(text):
    # Replace multiple newlines with a single newline
    cleaned_text = re.sub(r'\n+', '\n', text)

    # Prepend numbers between newlines to the next line
    cleaned_text = re.sub(r'\n(\d+)\n', r'\n\1 ', cleaned_text)

    return cleaned_text

In [151]:
%%time

import os
import pandas as pd
from bs4 import BeautifulSoup

def extract_info_from_html(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as file:
            content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    
    # Extract the main title
    main_title_container = soup.find(class_="in-article-module__container")
    main_title = main_title_container.find('h1').get_text(strip=True) if main_title_container and main_title_container.find('h1') else 'No Title Found'
    
    # Find all question-answer pairs
    q_and_a = []
    qa_containers = soup.find_all(class_="in-article-module__title-cont")
    for container in qa_containers:
        question_tag = container.find('h2')
        answer_tag = container.find_next_sibling(class_="in-article-module__section")

        question = question_tag.get_text(strip=True) if question_tag else 'No Question Found'
        answer = answer_tag.get_text(strip=False) if answer_tag else 'No Answer Found'
        
        answer = clean_text(answer)

        q_and_a.append((question, answer))
    
    return main_title, q_and_a

def create_dataframe_from_directory(directory):
    data = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.html'):
                file_path = os.path.join(root, file)
                main_title, q_and_as = extract_info_from_html(file_path)
                for question, answer in q_and_as:
                    data.append([file_path, main_title, question, answer])
            
            
#             break
    df = pd.DataFrame(data, columns=['Path', 'Heading', 'Question', 'Answer'])
    return df

# Example usage
directory = 'ee.co.uk/'  # Replace with your directory path
df = create_dataframe_from_directory(directory)
# print(df)
df

CPU times: user 5.98 s, sys: 12.2 ms, total: 5.99 s
Wall time: 5.99 s


Path                                            Heading                  Question                                             Answer
0             ee.co.uk/help/subscriptions/calm-app.html                                      Calm App Help  About the Calm App on EE  \nCalm is the leading mental health brand with...
1             ee.co.uk/help/subscriptions/calm-app.html                                      Calm App Help      One month free trial  \nTo enjoy a one month free trial text CALM to...
2             ee.co.uk/help/subscriptions/calm-app.html                                      Calm App Help           After the trial  \nThe Calm service is removed from your accoun...
3     ee.co.uk/help/billing-payments/payment-methods...                               Flex plans explained                                                                           \n
4     ee.co.uk/help/billing-payments/payment-methods...                               Flex plans explained      What are Flex plans?  \nEE Flex plans give you complete freedom: no ...
...                                                 ...                                                ...                       ...                                                ...
1183     ee.co.uk/help/orders/consumer-electronics.html  Electronics, accessories and SIM free handset ...          Delivery queries  \nHow do I track my delivery?\nOnce your order...
1184     ee.co.uk/help/orders/consumer-electronics.html  Electronics, accessories and SIM free handset ...             About returns  \nHow do I request to return an item / all ite...
1185     ee.co.uk/help/orders/consumer-electronics.html  Electronics, accessories and SIM free handset ...      Packaging Guidelines  \nWhen returning an item back to us, it's impo...
1186     ee.co.uk/help/orders/consumer-electronics.html  Electronics, accessories and SIM free handset ...           Product queries  \nWho do I contact if I have a query related t...
1187     ee.co.uk/help/orders/consumer-electronics.html  Electronics, accessories and SIM free handset ...    Help with other orders  \nHow to get help with your order\nOrder numbe...

[1188 rows x 4 columns]

In [154]:
df.to_csv('help_qna.csv')

In [155]:
df.to_pickle('help_qna.pk')

In [153]:
print(df['Path'].drop_duplicates().values)

['ee.co.uk/help/subscriptions/calm-app.html'
 'ee.co.uk/help/billing-payments/payment-methods/flex-plans-explained.html'
 'ee.co.uk/help/billing-payments/payment-methods/change-my-mobile-payment-menthod.html'
 'ee.co.uk/help/billing-payments/payment-methods/how-do-i-pay-my-broadband-or-tv-bill.html'
 'ee.co.uk/help/billing-payments/payment-methods/how-do-i-set-up-charge-to-bill-through-the-microsoft-store.html'
 'ee.co.uk/help/billing-payments/payment-methods/charging-purchases-to-your-ee-bill.html'
 'ee.co.uk/help/billing-payments/payment-methods/how-do-i-pay-my-phone-bill.html'
 'ee.co.uk/help/billing-payments/payment-methods/how-do-i-set-up-charge-to-bill-on-my-apple-device.html'
 'ee.co.uk/help/billing-payments/payment-methods/how-do-i-set-up-charge-to-bill-through-the-google-play-store.html'
 'ee.co.uk/help/billing-payments/payment-methods/Premium-Rate-Services-and-PSD2.html'
 'ee.co.uk/help/billing-payments/payment-methods/how-can-i-pay-my-ee-bills.html'
 'ee.co.uk/help/billing-p

In [149]:
print(df['Answer'].values[42])


Members of the public can opt out of receiving all emergency alerts, to do so search for emergency alerts in your phone’s settings. It is not possible to opt out of alerts by type of hazard, for example only receiving messages about flooding but not for industrial accidents.
We strongly recommend that people do not opt out of the service, as it is intended to warn you when lives are in danger. It is possible to opt-out but we would encourage you not to. See the website for more details.
Mobile network operator tests
To opt out of mobile network operator tests, search your settings for ‘emergency alerts’ and turn off Test alerts.
Android phones and tablets
 
To opt out, search your settings for ‘emergency alerts’ and turn off Severe alerts and Emergency alerts
 
On Huawei devices running EMUI 11 or older, search your settings for ‘emergency alerts’ and please turn off “Extreme threats”, “Severe threats” and “Show amber alerts”
 
If this does not work, contact your device manufacturer
i

In [140]:
df[df['Path'] == 'ee.co.uk/help/mobile/fix-problem/emergency-alert-explained.html']

Path                      Heading                                           Question                                             Answer
18  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?  What are Emergency Alerts and how will you rec...  \nEmergency Alerts are a public information sy...
19  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?  What is an Emergency Alert test and when is th...  \nThe Emergency Alerts system was announced by...
20  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?          What do the messages look and sound like?  \nEmergency Alerts appear on your device's hom...
21  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?  What should I do when I receive an Emergency A...  \nRead the content carefully. An Emergency Ale...
22  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?  What should I do if I receive a message while ...  \nYou MUST NOT hold a mobile phone while drivi...
23  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?                                   Why not use SMS?  \nEmergency Alerts offer a number of advantage...
24  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?              Is my privacy and security protected?  \nEmergency Alerts require no personal informa...
25  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?  Will all the Mobile Networks provide Emergency...  \nEmergency Alerts will be sent across all mob...
26  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?                      Do I need to download an app?  \nEmergency Alerts do not require an App to be...
27  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?                              Is it used elsewhere?  \nEmergency Alerts have been adopted by many c...
28  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?  Will Emergency Alerts be available in Wales, S...  \nEmergency Alerts will be available in Englan...
29  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?   Is there a charge to receive an Emergency Alert?  \nThere are no charges to receive Emergency Al...
30  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?      Will this new service affect my battery life?  \nNeither Emergency Alerts nor leaving the abi...
31  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?                 Can I reply to an Emergency Alert?  \nThe messages are one-way only. Check the con...
32  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?      What languages will messages be broadcast in?  \nThe primary language will be English. Sendin...
33  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?                     How many people will it reach?  \nWe want to make sure these alerts reach as m...
34  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?         Will Emergency Alerts work on all devices?  \nEmergency Alerts should be received on the m...
35  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?          How do I check if my phone is compatible?  \nThe following options will be visible when y...
36  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?   What if my phone can’t receive Emergency Alerts?  \nEmergency Alerts are just one of many tools ...
37  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?        Does it matter what network my phone is on?  \nEmergency Alerts will be sent across all net...
38  ee.co.uk/help/mobile/fix-problem/emergency-ale...  What is an Emergency Alert?  Are you using all the network types, like 2G, ...  \nThe initial roll out will b

In [147]:
df['Answer'].values[41:42]

array(['\nIt is possible to opt out of the system if you need your phone to stay concealed.\nAfter the test, once Emergency Alerts are fully operational their use cannot be predicted as they will be used in emergencies.\nThe choice is yours, but we strongly recommend that people do not opt out of the service, as it is intended to warn you when lives are in danger.\n'],
      dtype=object)

In [68]:
import os
import pandas as pd
from bs4 import BeautifulSoup

def extract_info_from_html(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as file:
            content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    
    # Debugging: Print soup to check if file is read correctly
    print("Soup of the file:", soup.prettify()[:500])  # Prints first 500 characters of the soup for debugging

    # Extract the main title
    main_title_tag = soup.find(id="in-article-module__container--unique")
    main_title = main_title_tag.get_text(strip=True) if main_title_tag else 'No Title Found'
    print("Main Title:", main_title)  # Debugging

    # Find all sections
    sections = soup.find_all(class_="in-article-section")
    print(f"Found {len(sections)} sections")  # Debugging

    # Extract questions and answers
    q_and_a = []
    for section in sections:
        question_tags = section.find_all(id=lambda x: x and x.startswith("article-title"))
        print(f"Found {len(question_tags)} question tags in a section")  # Debugging

        for question_tag in question_tags:
            question = question_tag.get_text(strip=True) if question_tag else 'No Question Found'
            answer_tag = question_tag.find_next_sibling(class_="in-article-module__section")
            answer = answer_tag.get_text(strip=True) if answer_tag else 'No Answer Found'

            q_and_a.append((question, answer))
    
    return main_title, q_and_a

def create_dataframe_from_directory(directory):
    data = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.html'):
                file_path = os.path.join(root, file)
                print("Processing file:", file_path)  # Debugging
                main_title, q_and_as = extract_info_from_html(file_path)
                for question, answer in q_and_as:
                    data.append([file_path, main_title, question, answer])

    df = pd.DataFrame(data, columns=['Path', 'Heading', 'Question', 'Answer'])
    return df

# Example usage
directory = 'ee.co.uk/help/mobile/fix-problem'  # Replace with your directory path
df = create_dataframe_from_directory(directory)
# print(df)


Processing file: ee.co.uk/help/mobile/fix-problem/i-have-a-problem-with-my-device.html
Soup of the file: ﻿
<!DOCTYPE html>
<!--[if IE 8]>         <html class="no-js lt-ie9" lang="en-gb" xmlns:og="http://ogp.me/ns#"> <![endif]-->
<!--[if IE 9]>         <html class="no-js ie9" lang="en-gb" xmlns:og="http://ogp.me/ns#"> <![endif]-->
<!--[if gt IE 9]> <html class="no-js gt-ie9" lang="en-gb" xmlns:og="http://ogp.me/ns#"> <![endif]-->
<!--[if !IE]><!-->
<html class="no-js" lang="en-gb">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>

Main Title: What should I do if I have a problem with my device?In this articleFind out what to do if your device is faulty, damaged, lost or stolen.What should I do if my device is faulty?What should I do if my device is faulty?Sometimes your device may stop working as a result of an internal fault. This could be related, but not limited, to a number of different causes such as; a software 

In [31]:
df

Empty DataFrame
Columns: [Path, Heading, Question, Answer]
Index: []